In [2]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 6.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.9 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.6/253.6 kB 15.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.11/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.11/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import json
import pymongo
from typing import List
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import AzureCosmosDBVectorSearch
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.schema.document import Document
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.messages import SystemMessage

In [6]:
load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
EMBEDDINGS_DEPLOYMENT_NAME = "embeddings"
COMPLETIONS_DEPLOYMENT_NAME = "completions"
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2023-09-01-preview"

In [7]:
llm = AzureChatOpenAI(            
        temperature = 0,
        openai_api_version = AOAI_API_VERSION,
        azure_endpoint = AOAI_ENDPOINT,
        openai_api_key = AOAI_KEY,         
        azure_deployment = "completions"
)
embedding_model = AzureOpenAIEmbeddings(
    openai_api_version = AOAI_API_VERSION,
    azure_endpoint = AOAI_ENDPOINT,
    openai_api_key = AOAI_KEY,   
    azure_deployment = "embeddings",
    chunk_size=10
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [8]:
pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.0/907.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.23.2
    Uninstalling openai-1.23.2:
      Successfully uninstalled openai-1.23.2

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.11/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_openai import AzureChatOpenAI

In [10]:
vector_store = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string = CONNECTION_STRING,
    namespace = "cosmic_works.products",
    embedding = embedding_model,
    index_name = "VectorSearchIndex",    
    embedding_key = "contentVector",
    text_key = "_id"
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_community/vectorstores/azure_cosmos_db.py:145: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client: MongoClient = MongoClient(connection_string, appname=appname)


In [11]:
query = "What yellow products are there?"
vector_store.similarity_search(query, k=3)

[Document(page_content='5B5E90B8-FEA2-4D6C-B728-EC586656FA6D', metadata={'categoryId': '75BF1ACB-168D-469C-9AA3-1FD26BB4EA4C', 'categoryName': 'Bikes, Touring Bikes', 'sku': 'BK-T79Y-60', 'name': 'Touring-1000 Yellow, 60', 'description': 'The product called "Touring-1000 Yellow, 60"', 'price': 2384.07, 'tags': [{'_id': '4F67013C-3B5E-4A3D-B4B0-8C597A491EB6', 'name': 'Tag-82'}, {'_id': '6C6D061E-F701-41DC-AEA2-7A5C28061840', 'name': 'Tag-98'}], 'contentVector': [-0.0038613835349678993, -0.02230239473283291, -0.04035269096493721, -0.018810605630278587, -0.02562522515654564, -0.006332387216389179, -0.0036431467160582542, 0.013382844626903534, 0.007708687335252762, -0.029792845249176025, 0.009123706258833408, -0.0381280817091465, -0.026737529784440994, -0.033340953290462494, 0.02680792845785618, 0.05040566250681877, -0.007363732438534498, -0.011404633522033691, -0.019739871844649315, 0.06409122049808502, 0.04578749090433121, -0.03243984654545784, 0.04865976795554161, 0.028384864330291748, 

In [13]:
system_prompt = """
You are a helpful, fun and friendly sales assistant for Cosmic Works, a bicycle and bicycle accessories store. 
Your name is Cosmo.
You are designed to answer questions about the products that Cosmic Works sells.

Only answer questions related to the information provided in the list of products below that are represented
in JSON format.

If you are asked a question that is not in the list, respond with "I don't know."

Only answer questions related to Cosmic Works products, customers, and sales orders.

If a question is not related to Cosmic Works products, customers, or sales orders,
respond with "I only answer questions about Cosmic Works"

List of products:
{products}

Question:
{question}
"""

In [14]:
def format_docs(docs:List[Document]) -> str:
        """
        Prepares the product list for the system prompt.
        """
        str_docs = []
        for doc in docs:
                # Build the product document without the contentVector
                doc_dict = {"_id": doc.page_content}
                doc_dict.update(doc.metadata)
                if "contentVector" in doc_dict:  
                        del doc_dict["contentVector"]
                str_docs.append(json.dumps(doc_dict, default=str))                  
        # Return a single string containing each product JSON representation
        # separated by two newlines
        return "\n\n".join(str_docs)

In [15]:
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create the prompt template from the system_prompt text
llm_prompt = PromptTemplate.from_template(system_prompt)

rag_chain = (
    # populate the tokens/placeholders in the llm_prompt 
    # products takes the results of the vector store and formats the documents
    # question is a passthrough that takes the incoming question
    { "products": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    # pass the populated prompt to the language model
    | llm
    # return the string ouptut from the language model
    | StrOutputParser()
)

In [16]:
question = "What are the names and skus of yellow products? Output the answer as a bulleted list."
response = rag_chain.invoke(question)
print(response)

- Touring-3000 Yellow, 58 (SKU: BK-T18Y-58)
- Touring-1000 Yellow, 50 (SKU: BK-T79Y-50)
- Touring-1000 Yellow, 60 (SKU: BK-T79Y-60)
- Road-550-W Yellow, 40 (SKU: BK-R64Y-40)


In [17]:
def create_cosmic_works_vector_store_retriever(collection_name: str, top_k: int = 3):
    vector_store =  AzureCosmosDBVectorSearch.from_connection_string(
        connection_string = CONNECTION_STRING,
        namespace = f"cosmic_works.{collection_name}",
        embedding = embedding_model,
        index_name = "VectorSearchIndex",    
        embedding_key = "contentVector",
        text_key = "_id"
    )
    return vector_store.as_retriever(search_kwargs={"k": top_k})


products_retriever = create_cosmic_works_vector_store_retriever("products")
customers_retriever = create_cosmic_works_vector_store_retriever("customers")
sales_retriever = create_cosmic_works_vector_store_retriever("sales")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_community/vectorstores/azure_cosmos_db.py:145: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client: MongoClient = MongoClient(connection_string, appname=appname)


In [18]:
# Create tools that will use vector search in vCore-based Azure Cosmos DB for MongoDB collections

# create a chain on the retriever to format the documents as JSON
products_retriever_chain = products_retriever | format_docs
customers_retriever_chain = customers_retriever | format_docs
sales_retriever_chain = sales_retriever | format_docs

tools = [
    Tool(
        name = "vector_search_products", 
        func = products_retriever_chain.invoke,
        description = "Searches Cosmic Works product information for similar products based on the question. Returns the product information in JSON format."
    ),
    Tool(
        name = "vector_search_customers", 
        func = customers_retriever_chain.invoke,
        description = "Searches Cosmic Works customer information and retrieves similar customers based on the question. Returns the customer information in JSON format."
    ),
    Tool(
        name = "vector_search_sales", 
        func = sales_retriever_chain.invoke,
        description = "Searches Cosmic Works customer sales information and retrieves sales order details based on the question. Returns the sales order information in JSON format."
    )
]

In [19]:
db = pymongo.MongoClient(CONNECTION_STRING).cosmic_works

def get_product_by_id(product_id: str) -> str:
    """
    Retrieves a product by its ID.    
    """
    doc = db.products.find_one({"_id": product_id})    
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc)

def get_product_by_sku(sku: str) -> str:
    """
    Retrieves a product by its sku.
    """
    doc = db.products.find_one({"sku": sku})
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc, default=str)

def get_sales_by_id(sales_id: str) -> str:
    """
    Retrieves a sales order by its ID.
    """
    doc = db.sales.find_one({"_id": sales_id})
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc, default=str)    

from langchain.tools import StructuredTool

tools.extend([
    StructuredTool.from_function(get_product_by_id),
    StructuredTool.from_function(get_product_by_sku),
    StructuredTool.from_function(get_sales_by_id)
])

/var/folders/ct/b2rs611d1g96lldn5x1gj1wm0000gp/T/ipykernel_12581/47035323.py:1: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  db = pymongo.MongoClient(CONNECTION_STRING).cosmic_works


In [20]:
system_message = SystemMessage(
    content = """
        You are a helpful, fun and friendly sales assistant for Cosmic Works, a bicycle and bicycle accessories store.

        Your name is Cosmo.

        You are designed to answer questions about the products that Cosmic Works sells, the customers that buy them, and the sales orders that are placed by customers.

        If you don't know the answer to a question, respond with "I don't know."
        
        Only answer questions related to Cosmic Works products, customers, and sales orders.
        
        If a question is not related to Cosmic Works products, customers, or sales orders,
        respond with "I only answer questions about Cosmic Works"
    """    
)
agent_executor = create_conversational_retrieval_agent(llm, tools, system_message = system_message, verbose=True)

In [21]:
result = agent_executor({"input": "What products do you have that are yellow?"})
print("***********************************************************")
print(result['output'])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `vector_search_products` with `yellow`


{"_id": "26E8185C-782A-4B48-87FA-1E715E3825FB", "categoryId": "AE48F0AA-4F65-4734-A4CF-D48B8F82267F", "categoryName": "Bikes, Road Bikes", "sku": "BK-R64Y-48", "name": "Road-550-W Yellow, 48", "description": "The product called \"Road-550-W Yellow, 48\"", "price": 1120.49, "tags": [{"_id": "46AC3482-E9A6-474D-A435-D3399F21991F", "name": "Tag-90"}, {"_id": "9C89E562-1247-435D-B786-4E54024E681C", "name": "Tag-128"}]}

{"_id": "91AA100C-D092-4190-92A7-7C02410F04EA", "categoryId": "F3FBB167-11D8-41E4-84B4-5AAA92B1E737", "categoryName": "Components, Touring Frames", "sku": "FR-T67Y-62", "name": "LL Touring Frame - Yellow, 62", "description": "The product called \"LL Touring Frame - Yellow, 62\"", "price": 333.42, "tags": [{"_id": "52FCE975-91EE-4789-9E36-94EC766F02A0", "name": "Tag-35"}]}

{"_id": "FFCA3096-199F-41C3-99D1-35BC88D8AC6F", "categoryId": "3E4CEACD-D007-46EB-82D7-31F6141752B2", "categoryN

In [22]:
result = agent_executor({"input": "What products were purchased for sales order '06FE91D2-B350-471A-AD29-906BF4EB97C4' ?"})
print("***********************************************************")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_sales_by_id` with `{'sales_id': '06FE91D2-B350-471A-AD29-906BF4EB97C4'}`


{"_id": "06FE91D2-B350-471A-AD29-906BF4EB97C4", "customerId": "D133397E-83ED-4FF3-BD44-62D7481237B2", "orderDate": "2011-07-01 00:00:00", "shipDate": "2011-07-08 00:00:00", "details": [{"sku": "LJ-0192-L", "name": "Long-Sleeve Logo Jersey, L", "price": 28.8404, "quantity": 1}, {"sku": "BK-R68R-52", "name": "Road-450 Red, 52", "price": 874.794, "quantity": 1}, {"sku": "CA-1098", "name": "AWC Logo Cap", "price": 5.1865, "quantity": 1}]}
Invoking: `get_product_by_sku` with `{'sku': 'LJ-0192-L'}`


{"_id": "6401B68F-924A-4B2E-AC9E-5660AEA0E848", "categoryId": "C3C57C35-1D80-4EC5-AB12-46C57A017AFB", "categoryName": "Clothing, Jerseys", "sku": "LJ-0192-L", "name": "Long-Sleeve Logo Jersey, L", "description": "The product called \"Long-Sleeve Logo Jersey, L\"", "price": 49.99, "tags": [{"_id": "B48D6572-67EB-4630-A1DB-AFD4AD7041C9", "name": "Tag-100"}, {"_id": "E1

In [23]:
result = agent_executor({"input": "What was the sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' ?"})
print("***********************************************************")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_sales_by_id` with `{'sales_id': '93436616-4C8A-407D-9FDA-908707EFA2C5'}`


{"_id": "93436616-4C8A-407D-9FDA-908707EFA2C5", "customerId": "6DF10977-EA10-4607-AF0F-74EDABA54943", "orderDate": "2013-09-30 00:00:00", "shipDate": "2013-10-07 00:00:00", "details": [{"sku": "BK-R89B-52", "name": "Road-250 Black, 52", "price": 1466.01, "quantity": 1}, {"sku": "HB-R956", "name": "HL Road Handlebars", "price": 72.162, "quantity": 1}, {"sku": "FR-R92B-44", "name": "HL Road Frame - Black, 44", "price": 858.9, "quantity": 3}, {"sku": "FR-R92R-44", "name": "HL Road Frame - Red, 44", "price": 858.9, "quantity": 1}]}The sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' is $4,114.12.

Let me know if there's anything else I can help with!

> Finished chain.
***********************************************************
The sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' is $4,114.12.

Let me know if ther

In [24]:
result = agent_executor({"input": "What was the price of the product with sku `FR-R92B-58` ?"})
print("***********************************************************")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_product_by_sku` with `{'sku': 'FR-R92B-58'}`


{"_id": "2CE4EFA7-5DC6-4D3E-ACB2-B7DDE4518408", "categoryId": "3E4CEACD-D007-46EB-82D7-31F6141752B2", "categoryName": "Components, Road Frames", "sku": "FR-R92B-58", "name": "HL Road Frame - Black, 58", "description": "The product called \"HL Road Frame - Black, 58\"", "price": 1431.5, "tags": [{"_id": "2CE9DADE-DCAC-436C-9D69-B7C886A01B77", "name": "Tag-101"}, {"_id": "AEFA79EF-CBF1-4824-AAF7-1D20EA85119B", "name": "Tag-17"}, {"_id": "D197C394-FB88-4EFF-B0FB-8BED1A86F294", "name": "Tag-182"}, {"_id": "ECBBCC15-3016-4075-B084-4B49DA754814", "name": "Tag-138"}]}The price of the product with SKU 'FR-R92B-58' is $1431.50.

Let me know if there's anything else I can assist you with!

> Finished chain.
***********************************************************
The price of the product with SKU 'FR-R92B-58' is $1431.50.

Let me know if there's anything else I can assist you with!
